# Description

Generates manubot tables for PhenomeXcan and eMERGE associations given an LV name (which is the only parameter that needs to be specified in the Settings section below).

# Modules loading

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import re
from pathlib import Path

import pandas as pd

from entity import Trait
import conf

# Settings

In [3]:
LV_NAME = "LV246"

In [4]:
assert (
    conf.MANUSCRIPT["BASE_DIR"] is not None
), "The manuscript directory was not configured"

OUTPUT_FILE_PATH = conf.MANUSCRIPT["CONTENT_DIR"] / "50.00.supplementary_material.md"
display(OUTPUT_FILE_PATH)
assert OUTPUT_FILE_PATH.exists()

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier_manuscript/content/50.00.supplementary_material.md')

In [5]:
# result_set is either phenomexcan or emerge
LV_FILE_MARK_TEMPLATE = "<!-- {lv}:{result_set}_traits_assocs:{position} -->"

In [6]:
TABLE_CAPTION = "Table: Significant trait associations of {lv_name} in {result_set_name}. {table_id}"

In [7]:
TABLE_CAPTION_ID = "#tbl:sup:{result_set}_assocs:{lv_name_lower_case}"

In [8]:
RESULT_SET_NAMES = {
    "phenomexcan": "PhenomeXcan",
    "emerge": "eMERGE",
}

# Load data

## PhenomeXcan LV-trait associations

In [9]:
input_filepath = Path(conf.RESULTS["GLS"] / "gls_phenotypes-combined-phenomexcan.pkl")
display(input_filepath)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/bases_data/base_orig/results/gls/gls_phenotypes-combined-phenomexcan.pkl')

In [10]:
phenomexcan_lv_trait_assocs = pd.read_pickle(input_filepath)

In [11]:
phenomexcan_lv_trait_assocs.shape

(5782, 8)

In [12]:
phenomexcan_lv_trait_assocs.head()

,part_k,cluster_id,phenotype,lv,coef,pvalue,pvalue_twosided,fdr
0,29,22,100002_raw-Energy,LV246,0.002401,0.424213,0.848425,0.680765
1,29,22,100002_raw-Energy,LV607,-0.006323,0.691499,0.617002,0.873350
2,29,22,100002_raw-Energy,LV612,-0.000822,0.525786,0.948428,0.763843
3,29,22,100002_raw-Energy,LV74,-0.006035,0.685313,0.629373,0.870769
4,29,22,100002_raw-Energy,LV838,0.024454,0.023446,0.046891,0.087403


## eMERGE LV-trait associations

In [13]:
input_filepath = Path(conf.RESULTS["GLS"] / "gls_phenotypes-combined-emerge.pkl")
display(input_filepath)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/bases_data/base_orig/results/gls/gls_phenotypes-combined-emerge.pkl')

In [14]:
emerge_lv_trait_assocs = pd.read_pickle(input_filepath)

In [15]:
emerge_lv_trait_assocs.shape

(7725, 6)

In [16]:
emerge_lv_trait_assocs.head()

,phenotype,lv,coef,pvalue,pvalue_twosided,fdr
0,008,LV246,-0.011008,0.809325,0.381350,0.999418
1,008,LV30,0.022387,0.038699,0.077397,0.483734
2,008,LV57,0.003960,0.379121,0.758243,0.961449
3,008,LV865,-0.008742,0.758337,0.483327,0.999418
4,008,LV847,-0.028246,0.985827,0.028346,0.999418


## eMERGE traits info

In [17]:
input_filepath = conf.EMERGE["DESC_FILE_WITH_SAMPLE_SIZE"]
display(input_filepath)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/bases_data/base_orig/data/emerge/eMERGE_III_PMBB_GSA_v2_2020_phecode_AFR_EUR_cc50_counts_w_dictionary.txt')

In [18]:
emerge_traits_info = pd.read_csv(
    input_filepath,
    sep="\t",
    dtype={"phecode": str},
    usecols=[
        "phecode",
        "phenotype",
        "category",
        "eMERGE_III_EUR_case",
        "eMERGE_III_EUR_control",
    ],
)

In [19]:
emerge_traits_info = emerge_traits_info.set_index("phecode")

In [20]:
emerge_traits_info = emerge_traits_info.rename(
    columns={
        "eMERGE_III_EUR_case": "eur_n_cases",
        "eMERGE_III_EUR_control": "eur_n_controls",
    }
)

In [21]:
emerge_traits_info.shape

(309, 4)

In [22]:
emerge_traits_info.head()

,eur_n_cases,eur_n_controls,phenotype,category
phecode,,,,
008,1639,57495,Intestinal infection,infectious diseases
008.5,1024,57495,Bacterial enteritis,infectious diseases
008.52,893,57495,Intestinal infection due to C. difficile,infectious diseases
038,3172,50610,Septicemia,infectious diseases
038.3,1361,50610,Bacteremia,infectious diseases


In [23]:
assert emerge_traits_info.index.is_unique

# Trait associations

## PhenomeXcan

In [24]:
from traits import SHORT_TRAIT_NAMES

In [25]:
result_set = "phenomexcan"

In [26]:
def get_trait_objs(phenotype_full_code):
    if Trait.is_efo_label(phenotype_full_code):
        traits = Trait.get_traits_from_efo(phenotype_full_code)
    else:
        traits = [Trait.get_trait(full_code=phenotype_full_code)]

    # sort by sample size
    return sorted(traits, key=lambda x: x.n_cases / x.n, reverse=True)


def get_trait_description(phenotype_full_code):
    traits = get_trait_objs(phenotype_full_code)

    desc = traits[0].description
    if desc in SHORT_TRAIT_NAMES:
        return SHORT_TRAIT_NAMES[desc]

    return desc


def get_trait_n(phenotype_full_code):
    traits = get_trait_objs(phenotype_full_code)

    return traits[0].n


def get_trait_n_cases(phenotype_full_code):
    traits = get_trait_objs(phenotype_full_code)

    return traits[0].n_cases


def num_to_int_str(num):
    if pd.isnull(num):
        return ""

    return f"{num:,.0f}"


def get_part_clust(row):
    return f"{row.part_k} / {row.cluster_id}"

In [27]:
lv_assocs = phenomexcan_lv_trait_assocs[
    (phenomexcan_lv_trait_assocs["lv"] == LV_NAME)
    & (phenomexcan_lv_trait_assocs["fdr"] < 0.05)
].sort_values("fdr")

In [28]:
with pd.option_context(
    "display.max_rows", None, "display.max_columns", None, "display.max_colwidth", None
):
    display(lv_assocs)

,part_k,cluster_id,phenotype,lv,coef,pvalue,pvalue_twosided,fdr
2238,29,16,MAGNETIC_CH2.DB.ratio,LV246,0.095366,2.183932e-14,4.367864e-14,1.052291e-12
2490,29,17,hypercholesterolemia,LV246,0.066665,3.672563e-08,7.345126e-08,7.778300e-07
1362,29,18,3143_raw-Ankle_spacing_width,LV246,0.059177,1.331584e-06,2.663168e-06,2.059020e-05
1452,29,18,4119_raw-Ankle_spacing_width_right,LV246,0.054647,6.822593e-06,1.364519e-05,9.026232e-05
2244,29,16,MAGNETIC_HDL.C,LV246,0.050872,2.096086e-05,4.192172e-05,2.458232e-04
1824,29,17,6153_1-Medication_for_cholesterol_blood_pressure_diabetes_or_take_exogenous_hormones_Cholesterol_lowering_medication,LV246,0.050766,2.179410e-05,4.358819e-05,2.535482e-04
1422,29,18,4100_raw-Ankle_spacing_width_left,LV246,0.048594,5.305968e-05,1.061194e-04,5.391759e-04
2442,29,11,fasting blood glucose measurement,LV246,0.043937,2.315192e-04,4.630383e-04,1.898786e-03
2016,29,17,6177_1-Medication_for_cholesterol_blood_pressure_or_diabetes_Cholesterol_lowering_medication,LV246,0.038355,1.024150e-03,2.048300e-03,6.736787e-03
96,29,28,102_raw-Pulse_rate_automated_reading,LV246,0.037216,1.376973e-03,2.753946e-03,8.607197e-03


In [29]:
lv_assocs = lv_assocs.assign(
    phenotype_desc=lv_assocs["phenotype"].apply(get_trait_description)
)

In [30]:
lv_assocs = lv_assocs.assign(n=lv_assocs["phenotype"].apply(get_trait_n))

In [31]:
lv_assocs = lv_assocs.assign(n_cases=lv_assocs["phenotype"].apply(get_trait_n_cases))

In [32]:
lv_assocs = lv_assocs.assign(coef=lv_assocs["coef"].apply(lambda x: f"{x:.3f}"))

In [33]:
lv_assocs = lv_assocs.assign(
    fdr=lv_assocs["fdr"].apply(lambda x: f"{x:.2e}".replace("-", "&#8209;"))
)

In [34]:
lv_assocs = lv_assocs.assign(n=lv_assocs["n"].apply(num_to_int_str))

In [35]:
lv_assocs = lv_assocs.assign(n_cases=lv_assocs["n_cases"].apply(num_to_int_str))

In [36]:
lv_assocs = lv_assocs.assign(part_clust="")  # lv_assocs.apply(get_part_clust, axis=1))

In [37]:
lv_assocs = lv_assocs.drop(columns=["phenotype"])

In [38]:
lv_assocs.shape

(24, 11)

In [39]:
lv_assocs = lv_assocs[["phenotype_desc", "n", "n_cases", "fdr"]]

In [40]:
lv_assocs = lv_assocs.rename(
    columns={
        "part_clust": "Partition / cluster",
        "lv": "Latent variable (LV)",
        #         "coef": r"$\beta$",
        "fdr": "FDR",
        "phenotype_desc": "Trait description",
        "n": "Sample size",
        "n_cases": "Cases",
    }
)

In [41]:
with pd.option_context(
    "display.max_rows", None, "display.max_columns", None, "display.max_colwidth", None
):
    display(lv_assocs)

,Trait description,Sample size,Cases,FDR
2238,CH2DB NMR,"24,154",,1.05e&#8209;12
2490,High cholesterol (self-reported),"361,141","43,957",7.78e&#8209;07
1362,Ankle spacing width,"206,589",,2.06e&#8209;05
1452,Ankle spacing width (right),"114,614",,9.03e&#8209;05
2244,HDL Cholesterol NMR,"19,270",,2.46e&#8209;04
1824,Cholesterol lowering medication,"193,148","24,247",2.54e&#8209;04
1422,Ankle spacing width (left),"114,630",,5.39e&#8209;04
2442,Fasting Glucose,"46,186",,1.90e&#8209;03
2016,Cholesterol lowering medication,"165,340","38,057",6.74e&#8209;03
96,"Pulse rate, automated reading","340,162",,8.61e&#8209;03


### Fill empty

In [42]:
if lv_assocs.shape[0] == 0:
    lv_assocs.loc[0, "Trait description"] = "No significant associations"
    lv_assocs = lv_assocs.fillna("")

### Save

In [43]:
# start
lv_file_mark_start = LV_FILE_MARK_TEMPLATE.format(
    result_set=result_set, lv=LV_NAME, position="start"
)
display(lv_file_mark_start)

# end
lv_file_mark_end = LV_FILE_MARK_TEMPLATE.format(
    result_set=result_set, lv=LV_NAME, position="end"
)
display(lv_file_mark_end)

'<!-- LV246:phenomexcan_traits_assocs:start -->'

'<!-- LV246:phenomexcan_traits_assocs:end -->'

In [44]:
new_content = lv_assocs.to_markdown(index=False, disable_numparse=True)

In [45]:
# add table caption
table_caption = TABLE_CAPTION.format(
    lv_name=LV_NAME,
    result_set_name=RESULT_SET_NAMES[result_set],
    table_id="{"
    + TABLE_CAPTION_ID.format(result_set=result_set, lv_name_lower_case=LV_NAME.lower())
    + "}",
)
display(table_caption)

'Table: Significant trait associations of LV246 in PhenomeXcan. {#tbl:sup:phenomexcan_assocs:lv246}'

In [46]:
new_content += "\n\n" + table_caption

In [47]:
full_new_content = (
    lv_file_mark_start + "\n" + new_content.strip() + "\n" + lv_file_mark_end
)

In [48]:
with open(OUTPUT_FILE_PATH, "r", encoding="utf8") as f:
    file_content = f.read()

In [49]:
new_file_content = re.sub(
    lv_file_mark_start + ".*?" + lv_file_mark_end,
    full_new_content,
    file_content,
    flags=re.DOTALL,
)

In [50]:
with open(OUTPUT_FILE_PATH, "w", encoding="utf8") as f:
    f.write(new_file_content)  # .replace("\beta", r"\beta"))

## eMERGE

In [51]:
result_set = "emerge"

In [52]:
lv_assocs = emerge_lv_trait_assocs[
    (emerge_lv_trait_assocs["lv"] == LV_NAME) & (emerge_lv_trait_assocs["fdr"] < 0.05)
].sort_values("fdr")

In [53]:
with pd.option_context(
    "display.max_rows", None, "display.max_columns", None, "display.max_colwidth", None
):
    display(lv_assocs)

,phenotype,lv,coef,pvalue,pvalue_twosided,fdr
1604,276.41,LV246,0.054395,0.000008,0.000016,0.002402
1575,276.4,LV246,0.053259,0.000012,0.000024,0.002971


In [54]:
lv_assocs = lv_assocs.assign(
    phenotype_desc=lv_assocs["phenotype"].apply(
        lambda x: emerge_traits_info.loc[x, "phenotype"]
    )
)

In [55]:
lv_assocs = lv_assocs.assign(
    n=lv_assocs["phenotype"].apply(
        lambda x: emerge_traits_info.loc[x, ["eur_n_cases", "eur_n_controls"]].sum()
    )
)

In [56]:
lv_assocs = lv_assocs.assign(
    n_cases=lv_assocs["phenotype"].apply(
        lambda x: emerge_traits_info.loc[x, "eur_n_cases"]
    )
)

In [57]:
lv_assocs = lv_assocs.assign(coef=lv_assocs["coef"].apply(lambda x: f"{x:.3f}"))

In [58]:
lv_assocs = lv_assocs.assign(
    fdr=lv_assocs["fdr"].apply(lambda x: f"{x:.2e}".replace("-", "&#8209;"))
)

In [59]:
lv_assocs = lv_assocs.assign(n=lv_assocs["n"].apply(num_to_int_str))

In [60]:
lv_assocs = lv_assocs.assign(n_cases=lv_assocs["n_cases"].apply(num_to_int_str))

In [61]:
lv_assocs = lv_assocs.rename(columns={"phenotype": "phecode"})

In [62]:
lv_assocs.shape

(2, 9)

In [63]:
lv_assocs = lv_assocs[["phecode", "phenotype_desc", "n", "n_cases", "fdr"]]

In [64]:
lv_assocs = lv_assocs.rename(
    columns={
        "lv": "Latent variable (LV)",
        #         "coef": r"$\beta$",
        "fdr": "FDR",
        "phecode": "Phecode",
        "phenotype_desc": "Trait description",
        "n": "Sample size",
        "n_cases": "Cases",
    }
)

In [65]:
with pd.option_context(
    "display.max_rows", None, "display.max_columns", None, "display.max_colwidth", None
):
    display(lv_assocs)

,Phecode,Trait description,Sample size,Cases,FDR
1604,276.41,Acidosis,"41,838","1,526",2.40e&#8209;03
1575,276.4,Acid-base balance disorder,"42,005","1,693",2.97e&#8209;03


### Fill empty

In [66]:
if lv_assocs.shape[0] == 0:
    lv_assocs.loc[0, "Phecode"] = "No significant associations"
    lv_assocs = lv_assocs.fillna("")

### Save

In [67]:
# start
lv_file_mark_start = LV_FILE_MARK_TEMPLATE.format(
    result_set=result_set, lv=LV_NAME, position="start"
)
display(lv_file_mark_start)

# end
lv_file_mark_end = LV_FILE_MARK_TEMPLATE.format(
    result_set=result_set, lv=LV_NAME, position="end"
)
display(lv_file_mark_end)

'<!-- LV246:emerge_traits_assocs:start -->'

'<!-- LV246:emerge_traits_assocs:end -->'

In [68]:
new_content = lv_assocs.to_markdown(index=False, disable_numparse=True)

In [69]:
# add table caption
table_caption = TABLE_CAPTION.format(
    lv_name=LV_NAME,
    result_set_name=RESULT_SET_NAMES[result_set],
    table_id="{"
    + TABLE_CAPTION_ID.format(result_set=result_set, lv_name_lower_case=LV_NAME.lower())
    + "}",
)
display(table_caption)

'Table: Significant trait associations of LV246 in eMERGE. {#tbl:sup:emerge_assocs:lv246}'

In [70]:
new_content += "\n\n" + table_caption

In [71]:
full_new_content = (
    lv_file_mark_start + "\n" + new_content.strip() + "\n" + lv_file_mark_end
)

In [72]:
with open(OUTPUT_FILE_PATH, "r", encoding="utf8") as f:
    file_content = f.read()

In [73]:
new_file_content = re.sub(
    lv_file_mark_start + ".*?" + lv_file_mark_end,
    full_new_content,
    file_content,
    flags=re.DOTALL,
)

In [74]:
with open(OUTPUT_FILE_PATH, "w", encoding="utf8") as f:
    f.write(new_file_content)  # .replace("\beta", r"\beta"))